### KRX 데이터정보시스템
- 매일 주식거래 내용

['일자', '종가', '등락률', '시가', '고가', '저가', '거래대금'] 등 수집

In [1]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

import datetime, time
from datetime import date

import pandas as pd
import numpy as np
import requests
import time
import os, sys

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [4]:
# driver = wd.Chrome(driver_name)
driver = wd.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020103'
# 개별종목 시세추이 data-menu-id: MDC0201020103
driver.get(main_url)
time.sleep(1)

In [ ]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT017_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) \
           > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
# driver.find_element_by_css_selector(css_sel).click()  # by_scc 형식이 없어짐.
driver.find_element(By.CSS_SELECTOR, css_sel).click()
time.sleep(1)

In [ ]:
def get_data_company(com_name, start_date, end_date):

    com_ticker = com_name[:6]
    # 회사이름 입력 Q 버튼
#     driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_0').click()
    driver.find_element(By.CSS_SELECTOR, '#btnisuCd_finder_stkisu0_0').click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
#     driver.find_element_by_id('searchText__finder_stkisu0_0').clear()
    driver.find_element(By.ID, 'searchText__finder_stkisu0_0').clear()
    time.sleep(1)
#     driver.find_element_by_id('searchText__finder_stkisu0_0').send_keys(com_name)
    driver.find_element(By.ID, 'searchText__finder_stkisu0_0').send_keys(com_name)
    time.sleep(2)

    # 검색 버튼 푸시
#     driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_0').click()
    driver.find_element(By.CSS_SELECTOR, '#searchBtn__finder_stkisu0_0').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(1)'
    #jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(3)
#     driver.find_element_by_css_selector(css_sel).click()

    element = WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), com_ticker))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    driver.find_element(By.CSS_SELECTOR, css_sel).click()
    time.sleep(2)

    return

In [ ]:
def set_date(start_date, end_date): # 일정 기간 데이터 취득
    # end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
#     driver.find_element_by_id('endDd').clear()
    driver.find_element(By.ID, 'endDd').clear()
#     driver.find_element_by_id('endDd').send_keys(end_date)
    driver.find_element(By.ID, 'endDd').send_keys(end_date)
    time.sleep(1)
#     driver.find_element_by_id('strdDd').clear()
    driver.find_element(By.ID, 'strdDd').clear()
#     driver.find_element_by_id('strdDd').send_keys(start_date)
    driver.find_element(By.ID, 'strdDd').send_keys(start_date)
    time.sleep(1)

In [ ]:
def get_data(start_date, end_date):
    
    column_name = ['date', 'close', 'change', 'close_cr', 'open', 'high', 'low', 
                  'vol_quantity', 'vol','total_amount', 'total_counts' ]
    # ['일자', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', 
    #                                 '거래대금', '시가총액', '상장주식수']
 
    start_str = start_date.strftime('%Y-%m-%d')
    ed_str = end_date.strftime('%Y-%m-%d')
    set_date(start_str, ed_str)

    # 테이블  취득 버튼 클릭 (우상귀)
    driver.find_element(By.ID, 'jsSearchButton').click()
    time.sleep(5)

    df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
    df.columns = column_name
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
    df_get = df[['date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']]
    
    return df_get

In [ ]:
def non_empty_index_df(df_input, start_date, end_date): # 토,일,공휴일등 거래가 없는 일자도 모두 포함
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_input.set_index('date', inplace=True)
    df_out = pd.DataFrame(columns = df_input.columns)
    df_out.insert(0, 'date', date_range_ts)
    df_out.set_index('date', inplace=True)
    df_out.update(df_input)
    df_out.reset_index(inplace=True)
    return df_out

In [23]:
def concat_df(df_o, df):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=['date'], keep='last', inplace=True)
#     df_o.drop_duplicates(subset=['date'], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[0]], inplace=True)
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

In [ ]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [ ]:
pkl_directory = 'data/company_pkl/'
for key, val in code.items():
    com_name = "/".join([key, val[0]])
    pkl_name= '{}_historical.pkl'.format(val[1])
    df_o = pd.read_pickle(pkl_directory + pkl_name)
    start_date = df_o['date'].iloc[len(df_o)-1]
    end_date = datetime.date.today()
    get_data_company(com_name, start_date, end_date)
    df_get = get_data(start_date, end_date)
    df_out = non_empty_index_df(df_get, start_date, end_date)
#   NaN to null string. cause NaN does not replace original values in the original datafram    
    df_o = concat_df(df_o, df_out) # append df to original df
    df_o.replace(np.nan, '', inplace=True)
    df_o.to_pickle(pkl_directory+pkl_name)

In [ ]:
pkl_directory = 'data/company_pkl/'
# pkl_name= 'ktng_historical--.pkl'
# df_n = pd.read_pickle(pkl_directory + pkl_name)
pkl_name= 'ktng_investors.pkl'
df_o = pd.read_pickle(pkl_directory + pkl_name)

In [ ]:
cn = df_o.columns
cn

In [ ]:
print("최근 몇일전까지 자료를 추가할까요?, ", end='')
n_days = int(input().strip())
today = datetime.date.today()
diff_days = datetime.timedelta(days=n_days) # 최근 n_days일 이내 데이터 취득
start_date = today - diff_days
end_date = today

In [ ]:
company_name = ['005930/삼성전자', '005380/현대차', '035420/NAVER', '033780/KT&G']
company_code = {'sec': '005930', 'hyunmotor': '005380', 'naver': '035420', 'ktng': '033780'}
# company_name = ['005380/현대차']
# company_code = {'hyunmotor': '005380'}

company_match = {'005930': '005930/삼성전자', '005380': '005380/현대차',
                 '035420': '035420/NAVER', '033780': '033780/KT&G'}

In [ ]:
for key, _ in company_code.items():
    filename = 'data/data_{}.csv'.format(key)
    df_company = pd.read_csv(filename)
    df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))

    columns_historical_data = df_company.columns[[0, -9, -8, -7, -6, -5, -4]]
    df_historical_data = df_company[columns_historical_data]

    df_historical_data = df_historical_data[df_historical_data['date'] <= '2023-02-02']

    pkl_directory = 'data/company_pkl/'
    pkl_name = '{}_historical.pkl'.format(key)
    df_historical_data.to_pickle(pkl_directory+pkl_name)

In [ ]:
for key, _ in company_code.items():
    filename = 'data/data_{}.csv'.format(key)
    df_company = pd.read_csv(filename)
    df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))

    columns_investors_data = df_company.columns[[0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
    df_investors_data = df_company[columns_investors_data]

    df_investors_data = df_investors_data[df_investors_data['date'] <= '2023-02-02']

    pkl_directory = 'data/company_pkl/'
    pkl_name = '{}_investors.pkl'.format(key)
    df_investors_data.to_pickle(pkl_directory+pkl_name)

In [ ]:
print("최근 몇일전까지 자료를 추가할까요?, ", end='')
n_days = int(input().strip())
today = datetime.date.today()
diff_days = datetime.timedelta(days=n_days) # 최근 n_days일 이내 데이터 취득
start_date = today - diff_days
end_date = today

In [ ]:
def append_data(company_code, company_match, com_df):
    for key, val in company_code.items():
        filename = 'data/{}_{}.csv'.format('data', key)
        df_company = pd.read_csv(filename)
        df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
        df_company.set_index('date', inplace=True)

        df_new = com_df[company_match[val]]
        df_new.set_index('date', inplace=True)     
        
    #     df_company.update(df_new, overwrite=True) : default
    #           : df_new 값이 NaN 이면 스킵하고 그렇지 않으면 df_company에 있는 모든 값을 대체
    #     df_company.update(df_new, overwrite=False) : df_company 값이 NaN인 요소에만 값 대체)
        df_company.update(df_new) # df_new 값이 NaN인 경우에만 제외하고 df_new의 모든 값으로 대체
        df_company.reset_index(inplace=True)
        df_company.to_csv(filename, index=False)
    return df_company

In [ ]:
def make_pickle(df, pkl_name):
    pkl_directory = 'data/company_pkl/'
    try:
        if not os.path.exists(pkl_directory):
            os.makedirs(pkl_directory)
    except OSError:
        print("Error: Failed to create the directory.")

    # 데이터 저장: ../data/spx.pkl
    df.to_pickle(pkl_directory+pkl_name)

In [ ]:
def read_pickle(pkl_name):
# 데이터 로드
    pkl_directory = 'data/company_pkl/'
    df = pd.read_pickle(pkl_directory+pkl_name)
    
    return df

In [ ]:
def update_pickle(df, pkl_name):
    df_o = read_pickle(pkl_name)
    df_o = concat_df(df_o, df)
    
    make_pickle(df_o, pkl_name)

In [ ]:
df_o = pd.read_pickle(pkl_directory + pkl_name)

In [ ]:
df_o.tail(10)

In [ ]:
dff = pd.read_pickle(pkl_directory+'sec_historical--.pkl')

In [ ]:
code['033780'][1]

In [ ]:
# 메인 데이터 취득
com_df = {}

for com_name in company_name:
    get_data_company(com_name, start_date, end_date)
    df_get = get_data(start_date, end_date)
    df_out = non_empty_index_df(df_get, start_date, end_date)
    com_df[com_name] = df_out.replace(np.nan, '') 
    # NaN to null string. cause NaN does not replace original values in the original datafram
#     com_df[com_name] = get_data()
    
df = append_data(company_code, company_match, com_df)

In [ ]:
# df_out.head()

### 투자자별 현황
 - 거래 정보와 코딩 구성등 형태는 똑 같으나, empty_datetime index 내용과, column selected 내용이 서로 달라서,
    별개의 루틴으로 구성함.

In [5]:
# 투자자별 URL로 변경
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203'
driver.get(main_url)
time.sleep(2)

In [6]:
column_name = ['financial', 'insurance', 'invtrust', 'privequity', 'bank', 'financeetc', 'pension',
              'institution', 'corporateetc', 'retail', 'foreigner', 'foreigneretc', 'total' ]

In [7]:
# Message: element not interactable Error 방지용. 클릭하기 위하여는 그 위치가 클릭할 수 있게 노출되어 있어야 함
# 투자자별 거래실적 버튼이 위치한 곳으로 화면 scroll 

# id가 jsOpenView_1 인 element 를 찾음
# stop_tag = driver.find_element_by_id('jsOpenView_1')
stop_tag = driver.find_element(By.ID, 'jsOpenView_1')

# jsOpenView_1 element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(stop_tag).perform()

In [8]:
# 투자자별 거래 실적 버튼 클릭
# driver.find_element_by_id('jsOpenView_1').click()
driver.find_element(By.ID, 'jsOpenView_1').click()
time.sleep(2)

In [9]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT023_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
# driver.find_element_by_css_selector(css_sel).click()
driver.find_element(By.CSS_SELECTOR, css_sel).click()
time.sleep(1)

In [10]:
def date_set(datei): # 하루 하루 데이터를 받아야 함.
# end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
#     driver.find_element_by_id('endDd').clear()
#     driver.find_element_by_id('endDd').send_keys(datei)
    driver.find_element(By.ID, 'endDd').clear()
    driver.find_element(By.ID, 'endDd').send_keys(datei)
    time.sleep(1)
#     driver.find_element_by_id('strtDd').clear()
#     driver.find_element_by_id('strtDd').send_keys(datei)
    driver.find_element(By.ID, 'strtDd').clear()
    driver.find_element(By.ID, 'strtDd').send_keys(datei)
    time.sleep(1)
    return

In [11]:
def push_button_1(): # 조회 button push
    xp = '/html/body/div[2]/section[2]/section/section/div/div[2]/form/div[1]/div/a' 
    # use full xpath to avoid 'Message: element not interactable' Error
#     driver.find_element_by_xpath(xp).click()
    driver.find_element(By.XPATH, xp).click()
    time.sleep(1) # 여유시간 배분
    css_sel = 'div.loading-bar-wrap.small' # 각기 다른 loading 페이지에서 공통적으로 사용됨
    element = WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, css_sel)))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    time.sleep(1) # 여유시간 배분
    return

In [12]:
index_name = ['investor', 'sell_quantity', 'buy_quantity', 'pure_buy_quantiry', 'sell', 'buy', 'pure_buy']
def get_daily_data(date_range):
    df_org = None
    for datei in date_range:
        date_set(datei)
        push_button_1()
        df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
        df.columns = index_name
        df_new = df[['investor', 'pure_buy']]
        df_new.set_index('investor', inplace=True)
        dft = df_new.T
        dft.columns = column_name
        dft.insert(0, "date", datetime.datetime.strptime(datei, "%Y%m%d"))
        dft.reset_index(drop=True, inplace=True)
        if df_org is None:
            df_org = dft.copy()
            continue
#         df_org = df_org.append(dft, ignore_index=True) # append will be depreciated
        df_org =pd.concat([df_org,dft], ignore_index=True)
        
    return df_org

In [13]:
def get_data_company_investor(com_name, start_date, end_date):
    
    com_ticker = com_name[:6]
    # 회사이름 입력 Q 버튼
#     driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_1').click()
    driver.find_element(By.CSS_SELECTOR, '#btnisuCd_finder_stkisu0_1').click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
#     driver.find_element_by_id('searchText__finder_stkisu0_1').clear()
    driver.find_element(By.ID, 'searchText__finder_stkisu0_1').clear()
    time.sleep(1)
#     driver.find_element_by_id('searchText__finder_stkisu0_1').send_keys(com_name)
    driver.find_element(By.ID, 'searchText__finder_stkisu0_1').send_keys(com_name)
    time.sleep(2)

    # 검색 버튼 푸시
#     driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_1').click()
    driver.find_element(By.CSS_SELECTOR, '#searchBtn__finder_stkisu0_1').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_1 > tbody > tr:nth-child(1) > td:nth-child(1)'
#     driver.find_element_by_css_selector(css_sel).click()

    element = WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), com_ticker))
    driver.find_element(By.CSS_SELECTOR, css_sel).click()
    time.sleep(2)
    
    date_range = convert_date(start_date, end_date)
    return get_daily_data(date_range)

In [14]:
def convert_date(start_date, end_date):
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    date_range = []
    for x in date_range_ts:
        date_range.append(datetime.datetime.strftime(x, "%Y%m%d"))
    return date_range

In [ ]:
def append_data_investor(company_code, company_match, com_df):
    for key, val in company_code.items():
        filename = 'data/{}_{}.csv'.format('data', key)
        df_company = pd.read_csv(filename)
        df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
        df_company.set_index('date', inplace=True)

        df = com_df[company_match[val]]
        df_select = df[['date', 'financial', 'insurance', 'invtrust', 'privequity', 'bank', 'financeetc', 
                        'pension', 'institution', 'corporateetc', 'retail', 'foreigner', 'foreigneretc' ]]
        df_select.set_index('date', inplace=True)
        df_company.update(df_select)
        df_company.reset_index(inplace=True)
        df_company.to_csv(filename, index=False)
    return df_company

In [26]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [27]:
pkl_directory = 'data/company_pkl/'
for key, val in code.items():
    com_name = "/".join([key, val[0]])
    pkl_name= '{}_investors.pkl'.format(val[1])
    df_o = pd.read_pickle(pkl_directory + pkl_name)
    start_date = df_o['date'].iloc[len(df_o)-1]
    end_date = datetime.date.today()
    df_out = get_data_company_investor(com_name, start_date, end_date)
    df_out = df_out[df_o.columns]
    df_o = concat_df(df_o, df_out) # append df to original df
    pkl_name= '{}_investors--.pkl'.format(val[1])
    df_o.to_pickle(pkl_directory+pkl_name)

In [17]:
df_out.head()

,date,financial,insurance,invtrust,privequity,bank,financeetc,pension,institution,corporateetc,retail,foreigner,foreigneretc,total
0,2023-02-02,-49788,5575,-3370,-23213,-29,722,-8072,-78176,57123,-303219,325146,-875,0
1,2023-02-03,-62371,4764,-13900,-42280,-1948,26,-12896,-128604,-3103,-110538,242389,-144,0
2,2023-02-04,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2023-02-05,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2023-02-06,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
df_out = df_out[df_o.columns]

In [22]:
df_out.head()

,date,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
0,2023-02-02,-303219,325146,-78176,-49788,-3370,-8072,-23213,-29,5575,722,57123,-875
1,2023-02-03,-110538,242389,-128604,-62371,-13900,-12896,-42280,-1948,4764,26,-3103,-144
2,2023-02-04,0,0,0,0,0,0,0,0,0,0,0,0
3,2023-02-05,0,0,0,0,0,0,0,0,0,0,0,0
4,2023-02-06,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
df_o.tail()

,date,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
1124,2023-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1125,2023-01-30,4877.0,102175.0,-43458.0,-2918.0,-22315.0,-4095.0,-8129.0,1775.0,-5374.0,-2403.0,-63444.0,-149.0
1126,2023-01-31,319625.0,-417343.0,94237.0,115646.0,-26610.0,-23456.0,24319.0,7087.0,-2709.0,-40.0,1847.0,1634.0
1127,2023-02-01,21846.0,5524.0,-32282.0,20145.0,3067.0,-21557.0,-37550.0,-185.0,3514.0,284.0,4537.0,375.0
1128,2023-02-02,-303219.0,325146.0,-78176.0,-49788.0,-3370.0,-8072.0,-23213.0,-29.0,5575.0,722.0,57123.0,-875.0


In [24]:
df_o = concat_df(df_o, df_out) 

In [25]:
df_o.tail()

,date,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
1128,2023-02-02,-303219.0,325146.0,-78176.0,-49788.0,-3370.0,-8072.0,-23213.0,-29.0,5575.0,722.0,57123.0,-875.0
1129,2023-02-03,-110538.0,242389.0,-128604.0,-62371.0,-13900.0,-12896.0,-42280.0,-1948.0,4764.0,26.0,-3103.0,-144.0
1130,2023-02-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1131,2023-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1132,2023-02-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 투자자별 메인 데이터 취득
com_df = {}

for com_name in company_name:
    df_out = get_data_company_investor(com_name, start_date, end_date)
#     com_df[com_name] = df_out.replace(np.nan, '') 
    com_df[com_name] = df_out
    
# 데이터 파일로 저장하기
df_inv = append_data_investor(company_code, company_match, com_df)

In [ ]:
driver.close()
driver.quit()

In [ ]:
pkl_directory = 'data/company_pkl/'
pkl_name = 'hyunmotor_investors.pkl'
df_in = pd.read_pickle(pkl_directory+pkl_name)

In [ ]:
df_in.tail()

In [28]:
pkl_directory = 'data/company_pkl/'
pkl_name= 'ktng_investors--.pkl'
df_n = pd.read_pickle(pkl_directory + pkl_name)
pkl_name= 'ktng_investors.pkl'
df_o = pd.read_pickle(pkl_directory + pkl_name)

In [29]:
df_n.tail(10)

,date,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
1128,2023-02-02,2138.0,7838.0,-9777.0,-2004.0,-2728.0,-4974.0,-196.0,-3.0,215.0,-87.0,-247.0,48.0
1129,2023-02-03,-3180.0,5100.0,-1873.0,-909.0,-1262.0,-1842.0,787.0,0.0,21.0,1332.0,-45.0,-1.0
1130,2023-02-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1131,2023-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1132,2023-02-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df_o.tail()

,date,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
1124,2023-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1125,2023-01-30,-6873.0,10845.0,-3989.0,-959.0,-1995.0,-418.0,-294.0,-308.0,90.0,-106.0,18.0,0.0
1126,2023-01-31,418.0,-1567.0,1154.0,3663.0,-623.0,-1541.0,-883.0,-3.0,11.0,530.0,-46.0,42.0
1127,2023-02-01,-56.0,788.0,-705.0,113.0,-931.0,-411.0,506.0,0.0,27.0,-8.0,-32.0,5.0
1128,2023-02-02,2138.0,7838.0,-9777.0,-2004.0,-2728.0,-4974.0,-196.0,-3.0,215.0,-87.0,-247.0,48.0
